# Working with missing data

In this section, we will discuss missing (also referred to as `NA`) values in cudf. cudf supports having missing values in all dtypes. These missing values are represented by `<NA>`. These values are also referenced as "null values".

## How to Detect missing values

To detect missing values, you can use `isna()` and `notna()` functions.

In [110]:
import cudf
import numpy as np
import pandas as pd

In [3]:
df = cudf.DataFrame({'a': [1, 2, None, 4], 'b':[0.1, None, 2.3, 17.17]})

In [4]:
df

,a,b
0,1,0.1
1,2,<NA>
2,<NA>,2.3
3,4,17.17


In [5]:
df.isna()

,a,b
0,False,False
1,False,True
2,True,False
3,False,False


In [6]:
df['a'].notna()

0     True
1     True
2    False
3     True
Name: a, dtype: bool

One has to be mindful that in Python (and NumPy), the nan's don’t compare equal, but None's do. Note that pandas/NumPy uses the fact that np.nan != np.nan, and treats None like np.nan.



In [7]:
None == None

True

In [9]:
np.nan == np.nan

False

So as compared to above, a scalar equality comparison versus a None/np.nan doesn’t provide useful information.



In [10]:
df['b'] == np.nan

0    False
1    False
2    False
3    False
Name: b, dtype: bool

## Float dtypes and missing data

Because ``NaN`` is a float, a column of integers with even one missing values is cast to floating-point dtype. However this doesn't happen by default.

By default if a ``NaN`` value is passed to `Series` constructor, it is treated as `<NA>` value. 

In [6]:
cudf.Series([1, 2, np.nan])

0       1
1       2
2    <NA>
dtype: int64

Hence to consider a ``NaN`` as ``NaN`` you will have to pass `nan_as_null=False` parameter into `Series` constructor.

In [7]:
cudf.Series([1, 2, np.nan], nan_as_null=False)

0    1.0
1    2.0
2    NaN
dtype: float64

## Datetimes

For `datetime64` types, cudf doesn't support having `NaT` values. Instead these values which are specific to numpy and pandas are considered as null values(`<NA>`) in cudf. The actual underlying value of `NaT` is `min(int64)` and cudf retains the underlying value when converting a cudf object to pandas object.



In [14]:
datetime_series = cudf.Series([pd.Timestamp("20120101"), pd.NaT, pd.Timestamp("20120101")])
datetime_series

0    2012-01-01 00:00:00.000000
1                          <NA>
2    2012-01-01 00:00:00.000000
dtype: datetime64[us]

In [15]:
datetime_series.to_pandas()

0   2012-01-01
1          NaT
2   2012-01-01
dtype: datetime64[ns]

## Inserting missing data

All dtypes support insertion of missing value by assignment. Any specific location in series can made null by assigning it to `None`.

In [16]:
series = cudf.Series([1, 2, 3, 4])

In [17]:
series

0    1
1    2
2    3
3    4
dtype: int64

In [18]:
series[2] = None

In [19]:
series

0       1
1       2
2    <NA>
3       4
dtype: int64

## Calculations with missing data

Null values propagate naturally through arithmetic operations between pandas objects.

In [22]:
df1 = cudf.DataFrame({'a':[1, None, 2, 3, None], 'b':cudf.Series([np.nan, 2, 3.2, 0.1, 1], nan_as_null=False)})

In [24]:
df2 = cudf.DataFrame({'a':[1, 11, 2, 34, 10], 'b':cudf.Series([0.23, 22, 3.2, None, 1])})

In [25]:
df1

,a,b
0,1,NaN
1,<NA>,2.0
2,2,3.2
3,3,0.1
4,<NA>,1.0


In [26]:
df2

,a,b
0,1,0.23
1,11,22.0
2,2,3.2
3,34,<NA>
4,10,1.0


In [27]:
df1 + df2

,a,b
0,2,NaN
1,<NA>,24.0
2,4,6.4
3,37,<NA>
4,<NA>,2.0


While summing the data, `NA` values will be treated as `0`.

In [32]:
df1['a'].sum()

6

Since `NA` values are treated as `0`, the mean would result to 2 in this case `(1 + 0 + 2 + 3 + 0)/5 = 2`

In [34]:
df1['a'].mean()

2.0

Cumulative methods like `cumsum` and `cumprod` ignore `NA` values by default.

In [38]:
df1['a'].cumsum()

0       1
1    <NA>
2       3
3       6
4    <NA>
Name: a, dtype: int64

To preserve `NA` values in cumulative methods, provide `skipna=False`.

In [40]:
df1['a'].cumsum(skipna=False)

0       1
1    <NA>
2    <NA>
3    <NA>
4    <NA>
Name: a, dtype: int64

## Sum/product of Null/nans

The sum of an empty or all-NA Series of a DataFrame is 0.

In [45]:
cudf.Series([np.nan], nan_as_null=False).sum()

0.0

In [46]:
cudf.Series([], dtype='float64').sum()

0.0

The product of an empty or all-NA Series of a DataFrame is 1.

In [47]:
cudf.Series([np.nan], nan_as_null=False).prod()

1.0

In [48]:
cudf.Series([], dtype='float64').prod()

1.0

## NA values in GroupBy

`NA` groups in GroupBy are automatically excluded. For example:

In [52]:
df1

,a,b
0,1,NaN
1,<NA>,2.0
2,2,3.2
3,3,0.1
4,<NA>,1.0


In [53]:
df1.groupby('a').mean()

,b
a,
1,NaN
2,3.2
3,0.1


## Filling missing values: fillna

`fillna()` can fill in `NA` values with non-NA data.

In [74]:
df1

,a,b
0,1,NaN
1,<NA>,2.0
2,2,3.2
3,3,0.1
4,<NA>,1.0


In [75]:
df1['b'].fillna(10)

0    NaN
1    2.0
2    3.2
3    0.1
4    1.0
Name: b, dtype: float64

## Filling with cudf Object

You can also fillna using a dict or Series that is alignable. The labels of the dict or index of the Series must match the columns of the frame you wish to fill. The use case of this is to fill a DataFrame with the mean of that column.

In [111]:
import cupy as cp
dff = cudf.DataFrame(cp.random.randn(10, 3), columns=list('ABC'))

In [80]:
dff.iloc[3:5, 0] = np.nan

In [81]:
dff.iloc[4:6, 1] = np.nan

In [82]:
dff.iloc[5:8, 2] = np.nan

In [83]:
dff

,A,B,C
0,-1.707688,-0.059041,-0.544009
1,0.776755,-0.945995,-0.209892
2,1.479391,-1.130165,-3.035125
3,NaN,1.129463,0.837218
4,NaN,NaN,0.276491
5,-1.418267,NaN,NaN
6,2.244496,0.640298,NaN
7,-0.860345,1.296649,NaN
8,0.044818,-1.019191,-2.271279
9,0.158028,0.263081,0.252276


In [84]:
dff.fillna(dff.mean())

,A,B,C
0,-1.707688,-0.059041,-0.544009
1,0.776755,-0.945995,-0.209892
2,1.479391,-1.130165,-3.035125
3,NaN,1.129463,0.837218
4,NaN,NaN,0.276491
5,-1.418267,NaN,NaN
6,2.244496,0.640298,NaN
7,-0.860345,1.296649,NaN
8,0.044818,-1.019191,-2.271279
9,0.158028,0.263081,0.252276


In [98]:
dff.fillna(dff.mean()[1:3])

,A,B,C
0,-1.707688,-0.059041,-0.544009
1,0.776755,-0.945995,-0.209892
2,1.479391,-1.130165,-3.035125
3,NaN,1.129463,0.837218
4,NaN,NaN,0.276491
5,-1.418267,NaN,NaN
6,2.244496,0.640298,NaN
7,-0.860345,1.296649,NaN
8,0.044818,-1.019191,-2.271279
9,0.158028,0.263081,0.252276


Same result as above, but is aligning the ‘fill’ value which is a Series in this case.


In [102]:
# TODO: Look into this again
dff.where(dff.notna(), dff.mean())

TypeError: zip argument #2 must support iteration

## Dropping axis labels with missing data: dropna

Missing data can be excluded using `dropna()`:



In [103]:
df1

,a,b
0,1,NaN
1,<NA>,2.0
2,2,3.2
3,3,0.1
4,<NA>,1.0


In [105]:
df1.dropna(axis=0)

,a,b
0,1,NaN
2,2,3.2
3,3,0.1


In [107]:
df1.dropna(axis=1)

,b
0,NaN
1,2.0
2,3.2
3,0.1
4,1.0


An equivalent `dropna()` is available for Series. 

In [109]:
df1['a'].dropna()

0    1
2    2
3    3
Name: a, dtype: int64